In [ ]:
# Analyze connected components

import glob
import os
import cv2

folder_path = '/Users/vkluzner/OneDrive/NeuralMorphology/Simulations_16bit_Size1024/output/ex7/evaluation/'
file_list = glob.glob(os.path.join(folder_path, '*_pred_bin.tif'))
print("Files with '_pred_bin.tif':")
for file in file_list:
    image = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_UNCHANGED)
    _, binary_image = cv2.threshold(image, 127, 1, cv2.THRESH_BINARY)
    #num_labels, labeled_image = cv2.connectedComponents(binary_image)
    #num_cc = num_labels - 1 # Subtract 1 to exclude the background component
    num_labels, labeled_image, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)
    num_connected_components = num_labels - 1
    num_cc = num_labels - 1 # Subtract 1 to exclude the background component
    if num_cc > 1:
            print(f'Number of connected components for {file} is: {num_cc}')

In [ ]:
# Infer image through tiling process

import cv2
import math
import numpy as np
import torch
import os
import sys

from sklearn.mixture import GaussianMixture

def get_tif_images(folder):
    tif_files = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(".tif"):
                tif_files.append(os.path.join(root, file))
    return tif_files

def extend_image(orig_image, target_size):

    height, width = orig_image.shape
    
    extended_height, extended_width = height, width
    start_y_coord, start_x_coord = 0, 0
    
    if height < target_size:
        extended_height = target_size
        start_y_coord = (extended_height - height) // 2
    if width < target_size:
        extended_width = target_size
        start_x_coord = (extended_width - width) // 2
    extended_image = np.zeros((extended_height, extended_width), dtype=orig_image.dtype)
    
    # Implement Gaussian Mixture for Background Analysis:
    pixels = orig_image.flatten().reshape(-1, 1)  # Reshape to (n_samples, 1)
    gmm = GaussianMixture(n_components=2, random_state=42)
    gmm.fit(pixels)
    means = gmm.means_.flatten()
    variances = gmm.covariances_.flatten()
    background_label = np.argmin(means)

    # Step 3: Generate Gaussian-distributed pixel values
    extended_image = np.random.normal(loc=means[background_label], scale=np.sqrt(variances[background_label]), size=(extended_height, extended_width))
    extended_image[start_y_coord : start_y_coord+height, start_x_coord : start_x_coord + width] = orig_image
    
    return extended_image, start_y_coord, start_x_coord


# Get the current working directory
current_path = os.getcwd()
print("Current Working Directory:", current_path)
# Get the parent directory
parent_path = os.path.dirname(current_path)
print("Parent Directory:", parent_path)

#sys.path.append(current_path)
sys.path.append(parent_path)

# Print the updated sys.path
print("Updated sys.path:")
for p in sys.path:
    print(p)

# Find available GPU
if torch.backends.mps.is_available(): # Check if PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
    print("MPS is available!")
    if torch.backends.mps.is_built():
        print("MPS (Metal Performance Shader) is built in!")    
    device = "mps"
elif torch.cuda.is_available(): # Check if PyTorch has access to CUDA (Win or Linux's GPU architecture)
    print("CUDA is available!")
    device = "cuda"
else:
    print("Only CPU is available!")
    device = "cpu"
print(f"Using device: {device}")

#check out kernel
print("Python executable:", sys.executable)
print("Python version:", sys.version)

output_path = "/Users/vkluzner/OneDrive/NeuralMorphology/Simulations_16bit_Size3334/output/ex7/"

from model.unet_att import UnetAttention
model = UnetAttention()
model = model.to(device)
ckpt = torch.load(output_path + "ckpt.pth", weights_only=False, map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()  # Set to evaluation mode

# Get the list of real images
folder_path = "/Users/vkluzner/OneDrive/NeuralMorphology/CIV_Developmental_Images/"
tif_images = sorted(get_tif_images(folder_path))

#tif_images = ['/Users/vkluzner/OneDrive/NeuralMorphology/CIV_Developmental_Images/24hr/Processed/24hrWt1_40x_20gfp_500ms_495z_frmbottomA3A4A5-MaxIP-A3.tif',
#              '/Users/vkluzner/OneDrive/NeuralMorphology/CIV_Developmental_Images/24hr/Processed/Oriented/24hrWt2_40x_20gfp_500ms_482z_frmlefttopA3A4A5001-MaxIP-A4-Ori.tif']

# Infer the image through tiling
T = 512 # Tile size
threshold = 0.69

for current_file in tif_images:
    print("Infering image: ", current_file)
    orig_image = cv2.imread(current_file, cv2.IMREAD_UNCHANGED)  # or IMREAD_COLOR if 3 channels
    orig_image = cv2.convertScaleAbs(orig_image, alpha=255.0 / orig_image.max()) / 255.
    height, width = orig_image.shape

    if height < T or width < T: # Image is too small to be tiled - extend it
        image, start_y_coord, start_x_coord = extend_image(orig_image, T)
    else: # Image is large enough to be tiled - no need to extend
        image = orig_image.copy()
    Im_y, Im_x = image.shape
    
    # Create tiling coordinates
    n_x = math.ceil(Im_x / T)
    X_coord = np.zeros(n_x, dtype=int)
    if n_x == 1:
        gap_x = 0
    else:
        gap_x = math.floor((T * n_x - Im_x) / (n_x - 1))
    gap_x_plus_one__amount = T * n_x - Im_x - gap_x * (n_x - 1)
    for i in range(1, n_x):
        if i <= gap_x_plus_one__amount:
            X_coord[i] = int(X_coord[i-1] + T - (gap_x + 1))
        else:
            X_coord[i] = int(X_coord[i-1] + T - gap_x)

    n_y = math.ceil(Im_y / T)
    Y_coord = np.zeros(n_y, dtype=int)
    if n_y == 1:
        gap_y = 0
    else:
        gap_y = math.floor((T * n_y - Im_y) / (n_y - 1))
    gap_y_plus_one__amount = T * n_y - Im_y - gap_y * (n_y - 1)
    for i in range(1, n_y):
        if i <= gap_y_plus_one__amount:
            Y_coord[i] = int(Y_coord[i-1] + T - (gap_y + 1))
        else:
            Y_coord[i] = int(Y_coord[i-1] + T - gap_y)

    pred_array = np.zeros((n_x * n_y, Im_y, Im_x), dtype=np.float32)
    for i in range(n_y):
        for j in range(n_x):
            tile = image[Y_coord[i]:(Y_coord[i] + T), X_coord[j]:(X_coord[j] + T)] # Crop the ROI
            # Start the infering process
            tile_flip_0 = cv2.flip(tile, 0)
            tile_flip_1 = cv2.flip(tile, 1)
            tile_flip__1 = cv2.flip(tile, -1)
            tile_stack = np.stack([tile, tile_flip_0, tile_flip_1, tile_flip__1])
            tile_torch = torch.tensor(tile_stack).unsqueeze(1).to(torch.float32).to(device)
            with torch.no_grad():
                pred, _, _, _ = model(tile_torch)
                pred = torch.sigmoid(pred)
                pred_ori, pred_flip_0, pred_flip_1, pred_flip__1 = pred
            pred_ori = pred_ori.cpu().numpy()
            pred_flip_0 = cv2.flip(pred_flip_0.cpu().numpy(), 0)
            pred_flip_1 = cv2.flip(pred_flip_1.cpu().numpy(), 1)
            pred_flip__1 = cv2.flip(pred_flip__1.cpu().numpy(), -1)
            tile_pred = np.mean([pred_ori, pred_flip_0, pred_flip_1, pred_flip__1], axis=0)
            pred_array[i * n_x + j, Y_coord[i]:(Y_coord[i] + T), X_coord[j]:(X_coord[j] + T)] = tile_pred

    # Averaging the result and getting "soft" prediction
    non_zero_mask = pred_array != 0  # Shape (n_x * n_y, img_height, img_width)
    non_zero_count = np.sum(non_zero_mask, axis=0)  # Shape (img_height, img_width)
    non_zero_count[non_zero_count == 0] = 1  # Prevent division by zero
    #non_zero_sum = np.sum(pred_array * non_zero_mask, axis=0)  # Shape (img_height, img_width)
    #pred_image = non_zero_sum / non_zero_count  # Shape (img_height, img_width)
    pred_image = np.max(pred_array * non_zero_mask, axis=0)
    
    if height < T or width < T: # Cut original size of image from extended image
        pred_image = pred_image[start_y_coord : start_y_coord+height, start_x_coord : start_x_coord+width]
        
    # Getting "hard" prediction
    pred_bin_image = pred_image.copy()
    pred_bin_image[pred_bin_image >= threshold] = 1
    pred_bin_image[pred_bin_image < threshold] = 0
    
    # Save original file to the output path
    #current_file = os.path.join(output_path + 'real_mean', os.path.basename(current_file))
    current_file = os.path.join(output_path + 'real_max', os.path.basename(current_file))
    orig_image = (orig_image * 255).astype(np.uint8)
    cv2.imwrite(current_file, orig_image)

    # Save "soft" prediction
    soft_prediction_file = current_file.replace(".tif", "_pred.tif")
    pred_image = (pred_image * 255).astype(np.uint8)
    cv2.imwrite(soft_prediction_file, pred_image)
    
    # Save "hard" (binary) prediction
    current_hard_prediction = current_file.replace(".tif", "_bin.tif")
    pred_bin_image = (pred_bin_image * 255).astype(np.uint8)
    cv2.imwrite(current_hard_prediction, pred_bin_image)
